In [150]:
import os
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DistilBertTokenizer, DataCollatorWithPadding, TFAutoModelForSequenceClassification
from datasets import load_dataset, DatasetDict
from transformers import create_optimizer
from transformers.keras_callbacks import KerasMetricCallback

def batch_predict_sentiment(texts, tokenizer, model, batch_size=10):
    total_texts = len(texts)
    predicted_sentiments = []

    for i in range(0, total_texts, batch_size):
        texts_batch = texts[i:i + batch_size]
        inputs = tokenizer(texts_batch, return_tensors="pt", padding=True, truncation=True)

        with torch.no_grad():
            outputs = model(**inputs)

        predicted_classes = torch.argmax(outputs.logits, dim=1).tolist()
        predicted_sentiments.extend(predicted_classes)

    return predicted_sentiments

def calculate_accuracy(predicted_sentiments, actual_labels):
    correct_predictions = sum(1 for pred, label in zip(predicted_sentiments, actual_labels) if pred == label)
    total_predictions = len(actual_labels)
    accuracy = correct_predictions / total_predictions * 100
    return accuracy

def predict_model(test_data, model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    batch_predicted_sentiments = batch_predict_sentiment(test_data['text'], tokenizer, model)
    actual_labels = test_data['label']

    accuracy = calculate_accuracy(batch_predicted_sentiments, actual_labels)
    print(f"The accuracy achieved for predictions using {model_name} is {accuracy:.2f}%")

def finetuned_predict_distilbert(data):
    tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
    model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

    def preprocess_function(examples):
        return tokenizer(examples["text"], truncation=True)

    tokenized_imdb = data.map(preprocess_function, batched=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

    # Rest of the function remains the same

def main():
    imdb = load_dataset("imdb")
    small_test_dataset = imdb["test"].shuffle(seed=42).select(range(200))

    # Predict using GPT2
    predict_model(small_test_dataset, "mnoukhov/gpt2-imdb-sentiment-classifier")

    # Predict using RoBERTa
    predict_model(small_test_dataset, "abhishek/autonlp-imdb-roberta-base-3662644")

    # Predict using fine-tuned DistilBERT
    small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(3000))])
    small_val_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(300))])
    small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(300))])

    small_dataset_dict = DatasetDict({
        'train': small_train_dataset,
        'validation': small_val_dataset,
        'test': small_test_dataset
    })

    finetuned_predict_distilbert(small_dataset_dict)

if __name__ == '__main__':
    main()

tokenizer_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

...

The accuracy achieved for predictions using abhishek/autonlp-imdb-roberta-base-3662644 is 93.00%


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]